Siniflandirma ve regresyon icin kullanilabilir
Robust(dayanikli) bir regresyon modelleme teknigidir

Amac , bir marjin aralgiina maksimum noktayi en kucuk hata ile alabilecek sekilde dogru ya da egriyi belirlemektir

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

from warnings import filterwarnings
filterwarnings('ignore')

### Model & Tahmin

In [10]:
df = pd.read_csv('Hitters.csv')
df = df.dropna()
dms= pd.get_dummies(df[['League','Division','NewLeague']]) #kategorik degiskenleri dummy degiskenlere donusturduk
y=df["Salary"]
X_ = df.drop(['Salary','League','Division','NewLeague'],axis=1).astype('float64')
X=pd.concat([X_,dms[['League_N','Division_W','NewLeague_N']]],axis=1)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [14]:
svr_model = SVR(kernel="linear").fit(X_train,y_train)

In [16]:
svr_model

SVR(kernel='linear')

In [18]:
svr_model.predict(X_train)[0:5]

array([219.32624795, 702.43038724, 623.20561122, 153.77539402,
       463.15190748])

In [24]:
svr_model.predict(X_test)[0:5]

array([679.14756648, 633.72885336, 925.68636794, 270.28467553,
       530.26651616])

In [26]:
svr_model.intercept_

array([-80.15195923])

In [28]:
svr_model.coef_

array([[ -1.21839036,   6.0960298 ,  -3.67574544,   0.14217078,
          0.51435925,   1.28388991,  12.55922517,  -0.08693754,
          0.46597181,   2.98259947,   0.5294452 ,  -0.79820801,
         -0.16015542,   0.30872799,   0.28842351,  -1.79560062,
          6.41868986, -10.74313786,   1.33374319]])

In [32]:
#test 
y_pred = svr_model.predict(X_test)
np.sqrt(mean_squared_error(y_test,y_pred))

370.04084355099735